Capstone Project - The Battle of Neighborhoods -

The Best Big Texas City Having Many German Restaurants as a Holiday Destination for Senior German Tourist Group 

A senior German tourist group from Frankfurt am Main, Germany wants to purchase a holiday packages from  our Tourism Company. 
All of them are over 65 years old. They want to travel to a big city in Texas but they are insisting on a city having many German restaurants.
This is an indispensable prerequisite for this tourist group. Now we will use some data and  make some analyses to determine the best option for our customers.


In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import folium # map rendering library 

print('Libraries imported.')

Libraries imported.


In [2]:
CLIENT_ID = 'ZKAKN3BGDI2CK0X1VW3NLXXMF1YPXVHAH0VCH11PBHVWAVST' # my Foursquare ID
CLIENT_SECRET = 'PWRBSPQ4UKUC1PCF0ZO3I5CHACR4QPNX2NAJ2RTQMOMDBVFV' # my Foursquare Secret
VERSION = '20200518'

print('My credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)


My credentails:
CLIENT_ID: ZKAKN3BGDI2CK0X1VW3NLXXMF1YPXVHAH0VCH11PBHVWAVST
CLIENT_SECRET:PWRBSPQ4UKUC1PCF0ZO3I5CHACR4QPNX2NAJ2RTQMOMDBVFV


For our analyses the most populated five Texas cities will be used. 
The information about the cities and venues (German restaurants) will be taken from foursquare.

In [3]:
# type the selected 5 most populated Texas cities and Foursquare API German restaurant category ID
LIMIT = 500 # Maximum is 100
cities = ["Houston, TX", 'San Antonio, TX', 'Dallas, TX', 'Austin, TX', 'Fort Worth, TX']
results = {}
for city in cities:
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&near={}&limit={}&categoryId={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        city,
        LIMIT,
        "4bf58dd8d48988d10d941735") # GERMAN RESTAURANT CATEGORY ID
    results[city] = requests.get(url).json()

In [4]:
df_venues={}
for city in cities:
    venues = json_normalize(results[city]['response']['groups'][0]['items'])
    df_venues[city] = venues[['venue.name', 'venue.location.address', 'venue.location.lat', 'venue.location.lng']]
    df_venues[city].columns = ['Name', 'Address', 'Lat', 'Lng']


/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


The Foursquare API Only gives us the nearest 100 venues in a city.
We can check their densities by looking at them.

In [5]:
maps = {}
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=11)

    # add markers to map
    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])  
    print(f"Total number of German restaurants in {city} = ", results[city]['response']['totalResults'])
    print("Showing Top 100")

Total number of German restaurants in Houston, TX =  36
Showing Top 100
Total number of German restaurants in San Antonio, TX =  25
Showing Top 100
Total number of German restaurants in Dallas, TX =  20
Showing Top 100
Total number of German restaurants in Austin, TX =  11
Showing Top 100
Total number of German restaurants in Fort Worth, TX =  8
Showing Top 100


Each of the most populated 5 Texas cities has fewer than 100 German restaurants.
Houston has the most number of German restaurants among them.

In [6]:
maps[cities[0]]

In [7]:
maps[cities[1]]

In [8]:
maps[cities[2]]

In [9]:
maps[cities[3]]

In [10]:
maps[cities[4]]

Houston and San Antonio are the most densely German restaurant cities in Texas. 
However Houston has some more advantages with its population and nearness to Gulf of Mexico. 
Now we can focus on the measure of this density in order to get better and more scientific result. 
For this, the mean location of the German restaurants  and then the average of the distance of the venues
to the mean coordinates will be calculated.


In [11]:
maps = {}
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=11)
    venues_mean_coor = [df_venues[city]['Lat'].mean(), df_venues[city]['Lng'].mean()] 
    # add markers to map
    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])
        folium.PolyLine([venues_mean_coor, [lat, lng]], color="green", weight=1.5, opacity=0.5).add_to(maps[city])
    
    label = folium.Popup("Mean Co-ordinate", parse_html=True)
    folium.CircleMarker(
        venues_mean_coor,
        radius=10,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(maps[city])

    print(city)
    print("Mean Distance from Mean coordinates")
    print(np.mean(np.apply_along_axis(lambda x: np.linalg.norm(x - venues_mean_coor),1,df_venues[city][['Lat','Lng']].values)))

Houston, TX
Mean Distance from Mean coordinates
0.08846559317499479
San Antonio, TX
Mean Distance from Mean coordinates
0.07862892909621566
Dallas, TX
Mean Distance from Mean coordinates
0.09104875940778105
Austin, TX
Mean Distance from Mean coordinates
0.08536131680291237
Fort Worth, TX
Mean Distance from Mean coordinates
0.12214078337882303


In [12]:
maps[cities[0]]

In [13]:
maps[cities[1]]

In [14]:
maps[cities[2]]

In [15]:
maps[cities[3]]

In [16]:
maps[cities[4]]

According to results, San Antonio seems the best option. However, according to Foursquare API information, Houston is the most densely German restaurant city in Texas. 
When we check the results and observe the  Houston map, we can easily see that four German restaurants are far far away from our center point.
These may have caused problem and change or manipulated the results. Therefore, these four restaurants will be removed from the analysis to get a more accurate result.
After we remove these four restaurants, Houston will be still the most densely German restaurant city in Texas. 


In [17]:
city = 'Houston, TX'
venues_mean_coor = [df_venues[city]['Lat'].mean(), df_venues[city]['Lng'].mean()] 

print(city)
print("Mean Distance from Mean Coordinates")
dists = np.apply_along_axis(lambda x: np.linalg.norm(x - venues_mean_coor),1,df_venues[city][['Lat','Lng']].values)
dists.sort()
print(np.mean(dists[:-4]))# Ignore the 4 farthest distance

Houston, TX
Mean Distance from Mean Coordinates
0.05726521085264694


According to new result, Houston became the best option. These four restaurants had significantly changed the result of our analysis.
Houston is the best destination for our senior German tourist group insisting on a destination having many German restaurants.

Now, we can inform our customers and make preparations and reservations for the holiday in Houston.

